In [1]:
import copy
from data_loading import utils, purged_group_time_series
from models import resnet, lightning_nn
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import torch.nn as nn
from torch.utils.data import DataLoader
from pytorch_lightning import loggers as pl_loggers
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
import pytorch_lightning as pl
import numpy as np
import torch
from sklearn.preprocessing import OrdinalEncoder
import pandas as pd
import lightgbm as lgb
import xgboost as xgb
import optuna
import joblib
from models.SupervisedAutoEncoder import SupAE, create_hidden_rep
import os
import catboost as cat

In [2]:
p_ae = {'dim_1': 675, 'dim_2': 400, 'dim_3': 224, 'hidden': 162,
         'activation': nn.ReLU, 'dropout': 0.2916447561918717, 'lr': 0.030272591341587315,
         'recon_loss_factor': 0.4447516076774931, 'batch_size': 1252, 'loss_sup_ae': nn.MSELoss,
         'loss_recon': nn.MSELoss,
         'embedding': True,
         'input_size':310,'output_size':1,'batch_size':4000}
#p_res = {'dim_1': 843, 'dim_2': 2637, 'dim_3': 1618, 'dim_4': 880, 'dim_5': 220, 'activation': nn.LeakyReLU, 'dropout': 0.453246718032545, 'lr': 0.04565788979670108, 'batch_size': 10836,'loss':nn.MSELoss,'embedding':True,'input_size':310,'output_size':1,'hidden_len':p_ae['hidden']}
#model_res = resnet.ResNet(input_size=p_res['input_size'],output_size=p_res['output_size'],params=p_res)
#model_res.load_state_dict(torch.load('./saved_models/trained/ResNet_state_dict.pth'))
model_ae = utils.load_model(path='./saved_models/trained/trained_ae.pth',p=p_ae,pl_lightning=False,model=SupAE)
models_dict = {'ResNet':[model_res,p_res],'ae':[model_ae,p_ae]}


NameError: name 'model_res' is not defined

In [30]:
models_dict = {'lgb':[clf_lgb,p_lgb]}
df_preds = utils.create_predictions(models=models_dict,ae=False)

In [31]:
df_preds = utils.create_prediction_file()

In [3]:
pred_path = f"{utils.get_data_path(root_dir='./data')}/predictions/"

In [ ]:
df = utils.load_data(root_dir='./data/',mode='test')

In [5]:
val_eras = df[df['data_type']=='validation']['era'].unique().tolist()

In [6]:
eras = os.listdir(pred_path)

In [7]:
eras = [f"era{era.replace('.csv','')}" for era in eras]

In [8]:
eras = [era for era in eras if era in val_eras]

In [9]:
eras =[era.replace('era','') for era in eras]

In [10]:
eras = [f'{era}.csv' for era in eras]

In [32]:
df_preds = pd.read_csv('./data/numerai_dataset_259/predictions/predictions.csv')

In [ ]:
df = df.merge(df_preds, on = 'id')

In [ ]:
df_preds = df_preds[['id','prediction']]
df_preds.to_csv('./data/numerai_dataset_259/predictions.csv',index=False)

In [35]:
df_preds

,id,prediction
0,n0003aa52cab36c2,0.495438
1,n000920ed083903f,0.494747
2,n0038e640522c4a6,0.512597
3,n004ac94a87dc54b,0.501078
4,n0052fe97ea0c05f,0.500134
...,...,...
1693041,nffe7b93a3b669a5,0.509144
1693042,nffeaf15f7ecd10b,0.496386
1693043,nfff26fc05c7d497,0.503823
1693044,nfff4cabd97debf2,0.507167


In [33]:
scores = df_preds.groupby('era').apply(scoring)
np.mean(scores)

KeyError: 'era'

In [13]:
df = pd.read_csv('./data/numerai_dataset_259/predictions.csv')

In [14]:
df

,id,prediction
0,n0003aa52cab36c2,0.524231
1,n000920ed083903f,0.513588
2,n0038e640522c4a6,0.535152
3,n004ac94a87dc54b,0.512500
4,n0052fe97ea0c05f,0.513177
...,...,...
1693041,nffe7b93a3b669a5,0.503227
1693042,nffeaf15f7ecd10b,0.509167
1693043,nfff26fc05c7d497,0.524319
1693044,nfff4cabd97debf2,0.526400


In [11]:
df_preds = df

In [4]:
df = utils.load_data(root_dir='./data/',mode='test')
df = df[df['data_type'] == 'validation']
df_test = df
df = utils.load_data(root_dir='./data/',mode='train')

In [4]:
df['era'][df['era'] == 'eraX'] = 'era999'

<ipython-input-4-d35d0c86b098>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['era'][df['era'] == 'eraX'] = 'era999'


In [5]:
data,target,features,era = utils.preprocess_data(df,ordinal=True)
data_test,target_test,features_test,era_test = utils.preprocess_data(df_test,ordinal=True)
"""
data_ = copy.deepcopy(data)
target_ = copy.deepcopy(target)
oe = OrdinalEncoder()
data = oe.fit_transform(data)
target=oe.fit_transform(target_.values.reshape(-1,1)).reshape(-1)
"""

'\ndata_ = copy.deepcopy(data)\ntarget_ = copy.deepcopy(target)\noe = OrdinalEncoder()\ndata = oe.fit_transform(data)\ntarget=oe.fit_transform(target_.values.reshape(-1,1)).reshape(-1)\n'

In [6]:
del df ,df_test

In [7]:
## Autoencodr training
data = np.concatenate([data,data_test],0)
target = np.concatenate([target,target_test],0)
era = np.concatenate([era,era_test],0)
t_idx =np.where(era <121)[0].tolist()
v_idx =np.where(era >=121)[0].tolist()

In [8]:
p_ae = joblib.load('./hpo/params/ae_sup_params.pkl')
p_ae = p_ae.best_params

In [9]:
p_ae

{'dim_1': 957,
 'dim_2': 973,
 'dim_3': 418,
 'hidden': 211,
 'activation': 'gelu',
 'dropout': 0.4359906639099919,
 'lr': 0.04371817065406649,
 'recon_loss_factor': 0.3347465392013046,
 'batch_size': 4098}

In [91]:
#p_ae = {'dim_1': 1014, 'dim_2': 682, 'dim_3': 247, 'hidden': 119, 'activation': 'silu', 'dropout': 0.1216503434767066, 'lr': 0.017815077087898312, 'recon_loss_factor': 0.4519573434211407, 'batch_size': 941}

In [11]:
p_ae['activation'] = nn.GELU
p_ae['loss_sup_ae']= nn.MSELoss
p_ae['loss_recon']= nn.MSELoss
p_ae['embedding']=True

In [12]:
dataset = utils.FinData(data = data,target=target,era=era)
dataloaders = utils.create_dataloaders(dataset,indexes={'train':t_idx,'val':v_idx},batch_size=p_ae['batch_size'])
p_ae['input_size'] = len(features)
p_ae['output_size'] = 1
model = SupAE(params=p_ae)
es = EarlyStopping(monitor='val_sup_loss',patience=10,min_delta=0.0005,mode='min')
trainer = pl.Trainer(max_epochs=100,gpus=1,callbacks=[es])

GPU available: True, used: True
TPU available: False, using: 0 TPU cores


In [13]:
trainer.fit(model,train_dataloader = dataloaders['train'],val_dataloaders=dataloaders['val'])
torch.save(model.state_dict(),f'./saved_models/trained/ae_state_dict.pth')

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name             | Type       | Params
------------------------------------------------
0 | loss_recon       | MSELoss    | 0     
1 | loss_sup_ae      | MSELoss    | 0     
2 | embedding_layers | ModuleList | 4.7 K 
3 | encoder          | Sequential | 2.3 M 
4 | regressor        | Sequential | 634   
5 | decoder          | Sequential | 2.3 M 
------------------------------------------------
4.7 M     Trainable params
0         Non-trainable params
4.7 M     Total params
18.611    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [22]:
data_dict = data_dict = {'data': data, 'target': target,
                                 'features': features, 'era': era}

In [14]:
x_tr, x_val = data_dict['data'][t_idx], data_dict['data'][v_idx]
y_tr, y_val = data_dict['target'][t_idx],data_dict['target'][v_idx]
d_tr = lgb.Dataset(x_tr,label=y_tr)
d_val = lgb.Dataset(x_val,label=y_val)
clf = lgb.train(p_lgb,d_tr,1000,valid_sets=[d_val],early_stopping_rounds=50,verbose_eval=True)
clf.save_model(f'./saved_models/trained/lgb.bin')

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.026486 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1550
[LightGBM] [Info] Number of data points in the train set: 501808, number of used features: 310
[LightGBM] [Info] Start training from score 0.499997
[1]	valid_0's l2: 0.0499474
Training until validation scores don't improve for 50 rounds
[2]	valid_0's l2: 0.049947
[3]	valid_0's l2: 0.0499466
[4]	valid_0's l2: 0.0499462
[5]	valid_0's l2: 0.0499458
[6]	valid_0's l2: 0.0499456
[7]	valid_0's l2: 0.0499454
[8]	valid_0's l2: 0.0499451
[9]	valid_0's l2: 0.0499445
[10]	valid_0's l2: 0.049944
[11]	valid_0's l2: 0.0499438
[12]	valid_0's l2: 0.0499434
[13]	valid_0's l2: 0.0499432
[14]	valid_0's l2: 0.049943
[15]	valid_0's l2: 0.0499427
[16]	valid_0's l2: 0.0499423
[17]	valid_0's l2: 0.0499421
[18]	valid_0's l2: 0.0499417
[19]	valid_0's l2:

In [14]:
data_dict = {'data': data, 'target': target,
                 'features': features, 'era': era}

In [20]:
p_ae['input_size'] = len(features)
p_ae['output_size'] = 1
model = utils.load_model('./saved_models/trained/ae_state_dict.pth',
                    p=p_ae, pl_lightning=False, model=SupAE)


In [16]:

models_dict = {'lgb':[clf,p_lgb]}

In [15]:
ae_output = create_hidden_rep(model, data_dict)
data_dict['hidden'],ae_preds = ae_output['hidden'], ae_output['preds']
data_dict['hidden_true'] = True
p_ae['hidden_len'] = data_dict['hidden'].shape[-1]

In [24]:
data_dict['data']

array([[0., 2., 1., ..., 4., 2., 3.],
       [0., 0., 0., ..., 0., 1., 4.],
       [1., 2., 1., ..., 0., 1., 3.],
       ...,
       [1., 4., 2., ..., 0., 3., 3.],
       [0., 3., 3., ..., 3., 2., 2.],
       [0., 2., 2., ..., 3., 1., 2.]])

In [16]:
dataset = utils.FinData(
            data=data_dict['data'], target=data_dict['target'], era=data_dict['era'], hidden=data_dict.get('hidden', None))
dataloaders = utils.create_dataloaders(
            dataset, indexes={'train': t_idx}, batch_size=p['batch_size'])

In [7]:
p_res = {'dim_1': 843, 'dim_2': 2637, 'dim_3': 1618, 'dim_4': 880, 'dim_5': 220, 'activation': nn.LeakyReLU, 'dropout': 0.453246718032545, 'lr': 0.04565788979670108, 'batch_size': 10836,'loss':nn.MSELoss,'embedding':True,'input_size':310,'output_size':1,'hidden_len':p_ae['hidden']}

In [8]:
model = resnet.ResNet(input_size = p['input_size'],output_size=p['output_size'],params=p)
es = EarlyStopping(monitor='val_mse',patience=10,min_delta=0.0005,mode='min')
trainer = pl.Trainer(max_epochs=100,gpus=1,callbacks=[es])
trainer.fit(model,train_dataloader=dataloaders['train'],val_dataloaders=dataloaders['val'])
torch.save(model.state_dict(),f'./saved_models/trained/ResNet_state_dict.pth')

NameError: name 'p' is not defined

In [18]:
#model = utils.load_model('./saved_models/trained/ResNet_state_dict.pth',
 #                   p=p, pl_lightning=False, model=resnet.ResNet)


<All keys matched successfully>

In [9]:
model_res = resnet.ResNet(input_size=p_res['input_size'],output_size=p_res['output_size'],params=p_res)
model_res.load_state_dict(torch.load('./saved_models/trained/ResNet_state_dict.pth'))
model_ae = utils.load_model(path='./saved_models/trained/trained_ae.pth',p=p_ae,pl_lightning=False,model=SupAE)
models_dict = {'ResNet':[model_res,p_res],'ae':[model_ae,p_ae]}


In [18]:
utils.create_predictions(models=models_dict)

FileNotFoundError: [Errno 2] No such file or directory: './data/numerai_dataset_258/predictions/132.csv'

In [16]:
p['input_size'] = len(features)
p['output_size'] = 1
train = True
if train:
    gts = purged_group_time_series.PurgedGroupTimeSeriesSplit(n_splits=5,group_gap=5)
    models=[]
    for i, (train_idx,val_idx) in enumerate(gts.split(data,groups=era)):
        dataset = utils.FinData(data=data, target=target, era=era)
        dataloaders = utils.create_dataloaders(
        dataset, indexes={'train': train_idx, 'val': val_idx}, batch_size=p['batch_size'])
        model = SupAE(p)
        es = EarlyStopping(monitor='val_loss', patience=10,
                            min_delta=0.005, mode='min')
        trainer = pl.Trainer(max_epochs=100,
                                gpus=1,
                                callbacks=[es])
        trainer.fit(
            model, train_dataloader=dataloaders['train'], val_dataloaders=dataloaders['val'])
        torch.save(model.state_dict(), f'saved_models/ae_fold_{i}_state_dict.pth')
        models.append(model)
else:
    models_nn = utils.load_model('./saved_models/AE',p=p,pl_lightning=False,model=SupAE)


2, train_loss=1.090]
Epoch 1:  42%|████▏     | 118/283 [00:08<00:11, 14.24it/s, loss=1.09, v_num=20, val_loss=0.972, val_sup_loss=0.0511, train_loss=1.090, sup_loss=0.0748, recon_loss=1.190]
Validating: 0it [00:00, ?it/s]
Epoch 1:  49%|████▉     | 139/283 [00:09<00:09, 15.09it/s, loss=1.09, v_num=20, val_loss=0.972, val_sup_loss=0.0511, train_loss=1.090, sup_loss=0.0748, recon_loss=1.190]
/home/james/.virtualenvs/Kaggle/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:52: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(*args, **kwargs)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name             | Type       | Params
------------------------------------------------
0 | loss_recon       | MSELoss    | 0     
1 | loss_sup_ae      | MSELoss    | 0     
2 | embedding_layers | ModuleList | 4.7 K 
3 | encoder          | Sequential | 1.3 M 
4 | regressor  

In [56]:
df_test = utils.load_data(root_dir='./data/',mode='test')
df_test = df_test[df_test['data_type'] == 'validation']
df = utils.load_data(root_dir='./data/',mode='train')
data,target,features,era = utils.preprocess_data(df,nn=True)
data_test,target_test,features_test,era_test = utils.preprocess_data(df_test,nn=True)


In [58]:
data = np.concatenate([data,data_test],0)
target = np.concatenate([target,target_test],0)
era = np.concatenate([era,era_test],0)
t_idx =np.where(era <121)[0].tolist()
v_idx =np.where(era >=121)[0].tolist()
data_dict = {'data': data, 'target': target,
                 'features': features, 'era': era}
#data = np.concatenate([data,data_dict['hidden']],1)


In [4]:
#data = data.astype('int')

In [82]:
features = np.linspace(start = 0, stop=data.shape[-1]-1,num=data.shape[-1],dtype='int')

In [92]:
model = cat.CatBoostRegressor(loss_function='RMSE',eval_metric='RMSE',iterations=1000,learning_rate=0.0025, random_seed=0, subsample=0.8,depth=16, task_type='GPU')
model.fit(X=data[t_idx],y=target[t_idx], eval_set=(data[v_idx],target[v_idx]),early_stopping_rounds=50, verbose_eval=1)

0:	learn: 0.2232421	test: 0.2234898	best: 0.2234898 (0)	total: 306ms	remaining: 5m 5s
1:	learn: 0.2232420	test: 0.2234897	best: 0.2234897 (1)	total: 315ms	remaining: 2m 37s
2:	learn: 0.2232133	test: 0.2234898	best: 0.2234897 (1)	total: 626ms	remaining: 3m 27s
3:	learn: 0.2232133	test: 0.2234899	best: 0.2234897 (1)	total: 637ms	remaining: 2m 38s
4:	learn: 0.2231877	test: 0.2234899	best: 0.2234897 (1)	total: 940ms	remaining: 3m 6s
5:	learn: 0.2231877	test: 0.2234899	best: 0.2234897 (1)	total: 946ms	remaining: 2m 36s
6:	learn: 0.2231589	test: 0.2234895	best: 0.2234895 (6)	total: 1.26s	remaining: 2m 58s
7:	learn: 0.2231587	test: 0.2234895	best: 0.2234895 (7)	total: 1.27s	remaining: 2m 37s
8:	learn: 0.2231306	test: 0.2234889	best: 0.2234889 (8)	total: 1.58s	remaining: 2m 53s
9:	learn: 0.2231292	test: 0.2234889	best: 0.2234889 (8)	total: 1.61s	remaining: 2m 39s
10:	learn: 0.2231292	test: 0.2234889	best: 0.2234889 (8)	total: 1.62s	remaining: 2m 25s
11:	learn: 0.2230972	test: 0.2234887	best: 0

In [93]:
preds = model.predict(data[v_idx])

In [26]:
#data_,_,_,_ = utils.preprocess_data(df,nn=True)
p_xgb = joblib.load('./hpo/params/xgb_hpo_2021-04-12.pkl').best_params
p_lgb = joblib.load('./hpo/params/lgb_hpo_2021-04-12.pkl').best_params
p_lgb['objective']= 'huber'
p_lgb['metric']= 'huber'
p_lgb['n_jobs']=-1
p_xgb['booster']='gbtree'
p_xgb['tree_method']='gpu_hist'
p_xgb['verbosity']=1
p_xgb['n_jobs']=10
p_xgb['objective'] = 'reg:pseudohubererror'
p_xgb['eval_metric']='rmse'
"""p = {'learning_rate': 0.050803514080008098,
         'max_leaves': 50,
         'bagging_fraction': 0.9011886437667906,
         'bagging_freq': 5,
         'feature_fraction': 0.7287921216266973,
         'min_data_in_leaf': 396,
         'lambda_l1': 0.02217696438630042,
         'lambda_l2': 0.03985756503899372,
         'boosting': 'gbdt',
         'max_depth': 100,
         'objective': 'regression',
         'metric': 'mse',
         'n_jobs':-1}
p_xgb = {'learning_rate': 0.00775963914833853, 'max_depth': 10, 'max_leaves': 43, 'subsample': 0.571736653601962, 'colsample_bytree': 0.7848536668456085, 'min_child_weight': 34, 'lambda': 0.19056674702246584, 'alpha': 0.061222309705225526,'objective':'reg:squarederror',
         'booster':          'gbtree',
         'tree_method':      'gpu_hist',
         'verbosity':        1,
         'n_jobs':           10,
         'eval_metric':      'rmse'}"""

models_gbm = {'xgb':[],'lgb':[]}
scores = {'xgb':[],'lgb':[]}
preds = {'xgb':[],'lgb':[]}
x_tr, x_val = data[t_idx], data[v_idx]
y_tr, y_val = target[t_idx],target[v_idx]
d_tr = lgb.Dataset(x_tr,label=y_tr)
d_val = lgb.Dataset(x_val,label=y_val)
clf_lgb= lgb.train(p_lgb,d_tr,1000,valid_sets=[d_val],early_stopping_rounds=50,verbose_eval=True)
d_tr = xgb.DMatrix(x_tr, label=y_tr)
d_val = xgb.DMatrix(x_val, label=y_val)
clf_xgb = xgb.train(p_xgb, d_tr, 1000, [
    (d_val, 'eval')], early_stopping_rounds=50, verbose_eval=True)
preds_lgb = clf_lgb.predict(x_val)
score_lgb = mean_squared_error(y_val,preds_lgb)
preds_xgb = clf_xgb.predict(d_val)
score_xgb = mean_squared_error(y_val, preds_xgb)


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.264762 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1550
[LightGBM] [Info] Number of data points in the train set: 501808, number of used features: 310
[LightGBM] [Info] Start training from score 0.499997
[1]	valid_0's huber: 0.0249734
Training until validation scores don't improve for 50 rounds
[2]	valid_0's huber: 0.0249731
[3]	valid_0's huber: 0.0249728
[4]	valid_0's huber: 0.0249723
[5]	valid_0's huber: 0.0249721
[6]	valid_0's huber: 0.0249718
[7]	valid_0's huber: 0.0249715
[8]	valid_0's huber: 0.0249712
[9]	valid_0's huber: 0.024971
[10]	valid_0's huber: 0.0249707
[11]	valid_0's huber: 0.0249705
[12]	valid_0's huber: 0.02497
[13]	valid_0's huber: 0.0249697
[14]	valid_0's huber: 0.0249694
[15]	valid_0's huber: 0.0249691
[16]	valid_0's huber: 0.0249689
[17]	valid_0's huber: 0.0249689
[18]	valid_0's huber: 0.0249688
[19]	valid_0's huber: 0.0249

In [27]:
preds = {'xgb':[],'lgb':[]}
models_gbm['lgb'].append(clf_lgb)
models_gbm['xgb'].append(clf_xgb)
scores['lgb'].append(score_lgb)
scores['xgb'].append(score_xgb)
preds['lgb'].append(preds_lgb)
preds['xgb'].append(preds_xgb)

In [17]:
def correlation(predictions, targets):
    ranked_preds = predictions.rank(pct=True, method="first")
    return np.corrcoef(ranked_preds, targets)[0, 1]


# convenience method for scoring
def scoring(df):
    return correlation(df['prediction'], df['target'])


# Payout is just the score cliped at +/-25%
def payout(scores):
    return scores.clip(lower=-0.25, upper=0.25)

In [8]:
val_target = target[v_idx]

In [38]:
preds_ae = ae_preds[v_idx].squeeze()

In [28]:
preds_median = np.mean([preds_xgb,preds_lgb],axis=0)

In [94]:
df_preds = pd.DataFrame({'pred':preds,'target':val_target,'era':era[v_idx]})

In [95]:
corr_per_era = df_preds.groupby('era')[['pred', 'target']].apply(
        lambda x: correlation(x['pred'], x['target']))

In [96]:
sharpe_ratio = corr_per_era.mean()/corr_per_era.std()

In [97]:
rolling_max = (corr_per_era +
                   1).cumprod().rolling(window=100, min_periods=1).max()
daily_values = (corr_per_era + 1).cumprod()
max_drawdown = (rolling_max - daily_values).max()

In [98]:
sharpe_ratio

0.5994183888492634

In [99]:
max_drawdown

0.09594917771034872

In [100]:
corr_per_era

era
121    0.021332
122    0.020814
123    0.043069
124    0.025774
125    0.026407
126    0.017629
127    0.014410
128    0.060124
129   -0.013057
130    0.027118
131    0.050449
132    0.056295
197   -0.014955
198   -0.001228
199   -0.039116
200   -0.003235
201   -0.008183
202    0.033003
203    0.009287
204    0.030738
205    0.002030
206    0.005624
207    0.042192
208    0.053453
209    0.048515
210   -0.015003
211   -0.044093
212    0.013495
dtype: float64

In [101]:
corr_per_era.mean()

0.016531712503909533

In [84]:
model.plot_partial_dependence(data[t_idx],features=features.tolist())

AssertionError: feature with idx 197 is not used in model

In [165]:
a = 0.0218 + 0.0238
b = 0.0218/a
c = 0.0238/a

In [44]:
1 - c

0.4780701754385964

In [ ]:
0.02381347835443424

In [ ]:
del data_,df,data

In [ ]:
input_size = data
output_size= 1
p = {'dim_1': 1500,
    'dim_2': 1000,
    'dim_3': 500,
    'dim_4': 250,
    'dim_5': 150,
    'activation': nn.SiLU,
    'dropout': 0.2,
    'lr': 0.05,
    'label_smoothing':0.1,
    'amsgrad': False,
    'batch_size': 5000,
    'loss':nn.MSELoss(),
    'embedding':True}

In [ ]:
dataset = utils.FinData(data=data, target=target, era=era)
del df

In [ ]:
model = models_nn[-1]
model.eval().to('cuda')
index = np.linspace(0,dataset.data.shape[0],dataset.data.shape[0],dtype='int')

In [ ]:
batch_size = 5000
dataloaders = utils.create_dataloaders(dataset,{'train':index.tolist()},batch_size=batch_size)

In [ ]:
hiddens =[]
for i, batch in enumerate(dataloaders['train']):
    _,hidden,_,_ = model(batch['data'].view(batch['data'].size(1),-1).to('cuda'))
    hiddens.append(hidden.cpu().detach().numpy())

In [ ]:
hiddens = np.array([hiddens[i][j] for i in range(len(hiddens)) for j in range(len(hiddens[i]))])

In [ ]:
data  = np.concatenate([data,hiddens],axis=1)

In [ ]:
gts = purged_group_time_series.PurgedGroupTimeSeriesSplit(n_splits=5,group_gap=5)
dataset = utils.FinData(data=data, target=target, era=era)
models=[]
for i, (train_idx,val_idx) in enumerate(gts.split(data,groups=era)):
    
    dataloaders = utils.create_dataloaders(
    dataset, indexes={'train': train_idx, 'val': val_idx}, batch_size=p['batch_size'])
    model = resnet.ResNet(input_size=input_size,output_size=output_size,params=p)
    #model.apply(lambda x: utils.init_weights(x,'relu'))
    es = EarlyStopping(monitor='val_mse', patience=10,
                        min_delta=0.005, mode='min')
    trainer = pl.Trainer(max_epochs=100,
                            gpus=1,
                            callbacks=[es])
    trainer.fit(
        model, train_dataloader=dataloaders['train'], val_dataloaders=dataloaders['val'])
    torch.save(model.state_dict(), f'saved_models/Resnet/fold_{i}_state_dict.pth')
    models.append(model)



In [ ]:
df = utils.load_data(root_dir='./data/',mode='test')

In [ ]:
data,target,features,era = utils.preprocess_data(df.iloc[:10000,],test=True,ordinal=True)
data_,_,_,_=utils.preprocess_data(df.iloc[:10000,],test=True,nn=True)
"""
data = data[0:1000]
oe = OrdinalEncoder()
data = oe.fit_transform(data)
"""

In [ ]:
t = torch.tensor()

In [ ]:
data = torch.LongTensor(data)

In [ ]:
data.shape

In [ ]:
def correlation(predictions, targets):
    ranked_preds = predictions.rank(pct=True, method="first")
    return np.corrcoef(ranked_preds, targets)[0, 1]


# convenience method for scoring
def scoring(df):
    return correlation(df['preds'], df['target'])


# Payout is just the score cliped at +/-25%
def payout(scores):
    return scores.clip(lower=-0.25, upper=0.25)

In [ ]:
preds = []
for model in models_nn:
    model.eval()
    preds.append(model(data))


In [ ]:
predictions = [preds[i][1].detach().numpy() for i in range(len(preds))]

In [ ]:
predictions = np.mean(predictions,axis=0)

In [ ]:
predictions = predictions.reshape(-1)

In [ ]:
df_preds = pd.DataFrame.from_dict({'era':era,'preds':predictions,'target':target.T})


In [ ]:
corrs = df_preds.groupby('era').apply(scoring)
corrs

In [ ]:
payout(corrs).mean()


In [ ]:
predictions = models_nn[0](data).reshape(-1).detach().numpy()

In [ ]:
preds = []
for model in models_gbm:
    preds.append(model.predict(data_))

In [ ]:
predictions_gbm = np.mean(preds,axis=0)
df_gbm = pd.DataFrame.from_dict({'era':era,'preds':predictions_gbm,'target':target.T})

In [ ]:
corrs_gbm = df_gbm.groupby('era').apply(scoring)
corrs_gbm

In [ ]:
payout(corrs_gbm).mean()

In [ ]:
df[df['data_type']=='test']

In [ ]:

wojhed-riDni0-hopnok